In [1]:
%matplotlib inline

In [2]:
import numpy as np
import scipy.io as spio
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from scipy.ndimage import gaussian_filter1d
from scipy import stats

In [3]:
sns.set_style('ticks')
sns.set_context('talk')

In [4]:
data_dir = '/home/julia/data/ict/'

In [5]:
def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    
    from: `StackOverflow <http://stackoverflow.com/questions/7008608/scipy-io-loadmat-nested-structures-i-e-dictionaries>`_
    '''
    data = spio.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return data      

def todict(matobj):
    '''
    A recursive function which constructs from matobjects nested dictionaries
    '''
    dict = {}
    for strg in matobj._fieldnames:
        elem = matobj.__dict__[strg]
        if isinstance(elem, spio.matlab.mio5_params.mat_struct):
            dict[strg] = _todict(elem)
        else:
            dict[strg] = elem
    return dict

## Organize data per mouse

In [6]:
df = pd.read_csv(data_dir+"hf2.csv", index_col=0)
df = df[df['quality']==1]
df = df[df['licks']==1]
df = df[df['finalTimes']==1]

# Excludes 4 datasets based on quality, availability of lick data and using final time periods

In [13]:
mice = ['SHA', 'TAY', 'UUU', 'VVV', 'WEY', 'YOU']
for m in mice:
 
    df_mouse = pd.DataFrame(columns=['dataset', 'day', 'inf',
                                     'num', 'type', 'odor', 'performance', 
                                     'start', 'end',
                                     'odor_start', 'iti_start', 
                                     'licks', 'gpmt', 'rpmt',
                                     'emg', 'body', 'neck'])
    
    mouse_files = df[df['fnames'].str.contains(m)]['fnames']
    for f in mouse_files:
        mat = loadmat(data_dir+"orig/{}.mat".format(f))
        trials = [todict(mat["lick_data"][i]) for i in range(len(mat["lick_data"]))]
        
        for t in trials:
            t['dataset'] = f
            t['day'] = df[df['fnames']==f]['DfromRev'].values[0]
            t['inf'] = df[df['fnames']==f]['inf'].values[0]
            
            # change arrays to lists to fit in df
            for i in ['licks', 'gpmt', 'rpmt', 'emg', 'body', 'neck']:
                t[i] = [t[i]]
            df_mouse= df_mouse.append(pd.DataFrame.from_dict(t), sort=True)
        
    df_mouse = df_mouse.reset_index(drop=True)
    df_mouse.to_pickle(data_dir+"mice/{}.pkl".format(m))
    
    df_mouse_small = df_mouse.filter(['dataset', 'day', 'inf', 'num', 'type', 'odor', 'performance',
                                      'start', 'end', 'odor_start', 'iti_start'])
    df_mouse_small.to_csv(data_dir+"mice/{}_small.csv".format(m))

## Make one dataframe containing per trial data of all mice

In [15]:
df_all = pd.DataFrame(columns=['dataset', 'day', 'inf', 'num', 'type', 'odor', 'performance',
                                'start', 'end', 'odor_start', 'iti_start'])

for m in mice:
    df_mouse = pd.read_csv(data_dir+"mice/{}_small.csv".format(m), index_col=0)
    df_all = df_all.append(df_mouse)

df_all = df_all.reset_index(drop=True)
df_all.to_csv(data_dir+"all_mice.csv")

## Align licks and photometry to odor

In [ ]:
mice = ['SHA', 'TAY', 'UUU', 'VVV', 'WEY', 'YOU']

for i in ['gpmt']:# ['licks', 'gpmt', 'rpmt']:
 
    df_all_odor = pd.DataFrame(index=np.arange(-5000, 15000))
    for m in mice:
        
        df_mouse = pd.read_pickle(data_dir+"mice/{}.pkl".format(m)).filter(['odor_start', i])
        ts_odor = []
        for row in range(len(df_mouse)):
            ts = df_mouse.loc[row, i]
            zero_idx = df_mouse.loc[row, 'odor_start']
            ts_odor.append(pd.Series(name="{}_{}".format(m,row), data=ts,
                                    index=np.arange(-zero_idx, -zero_idx+ts.shape[0])))
        df_odor = pd.DataFrame(index=np.arange(-5000, 15000))        
        df_odor = df_odor.join(ts_odor, how='left')
        np.save("/home/julia/data/ict/odor_aligned/{}_{}.npy".format(m, i), np.asarray(df_odor))

        df_all_odor = df_all_odor.join(df_odor, how='left')
    np.save("/home/julia/data/ict/odor_aligned/all_{}.npy".format(i), np.asarray(df_all_odor))

## Align licks and photometry to ITI

In [7]:
mice = ['SHA', 'TAY', 'UUU', 'VVV', 'WEY', 'YOU']

for i in ['licks']: #, ['licks', 'gpmt', 'rpmt']:
 
    df_all_iti = pd.DataFrame(index=np.arange(-10000, 10000))
    for m in mice:
        
        df_mouse = pd.read_pickle(data_dir+"mice/{}.pkl".format(m)).filter(['iti_start', i])
        ts_iti = []
        for row in range(len(df_mouse)):
            ts = df_mouse.loc[row, i]
            zero_idx = df_mouse.loc[row, 'iti_start']
            ts_iti.append(pd.Series(name="{}_{}".format(m,row), data=ts,
                                    index=np.arange(-zero_idx, -zero_idx+ts.shape[0])))
        df_iti = pd.DataFrame(index=np.arange(-10000, 10000))        
        df_iti = df_iti.join(ts_iti, how='left')
        np.save("/home/julia/data/ict/iti_aligned/{}_{}.npy".format(m, i), np.asarray(df_iti))

        df_all_iti = df_all_iti.join(df_iti, how='left')
    np.save("/home/julia/data/ict/iti_aligned/all_{}.npy".format(i), np.asarray(df_all_iti))